In [54]:
# TODO support layers, units
import ray
import dni
import importlib
importlib.reload(dni)
import numpy as np
ray.shutdown()
ray.init(address="10.128.0.51:6379")

2019-11-25 20:58:37,937	WARNING worker.py:1268 -- WARNING: Not updating worker name since `setproctitle` is not installed. Install this with `pip install setproctitle` (or ray[debug]) to enable monitoring of worker processes.


{'node_ip_address': '10.128.0.51',
 'object_store_address': '/tmp/ray/session_2019-11-24_05-06-15_858496_1204/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2019-11-24_05-06-15_858496_1204/sockets/raylet',
 'redis_address': '10.128.0.51:6379',
 'session_dir': '/tmp/ray/session_2019-11-24_05-06-15_858496_1204',
 'webui_url': None}

In [151]:
class ActivationExt(dni.model.LocalActivationExt):
    def preprocess(self, input):
        seq = input
        char2int = {' ': 72, '$': 23, "'": 12, '(': 68, ')': 8, '*': 71, '+': 33, ',': 24, '-': 26, '.': 2,
         '/': 5, '0': 22, '1': 6, '2': 43, '3': 38, '4': 14, '5': 58, '6': 66, '7': 18, '8': 55, '9': 69,
         '<': 52, '=': 57, '>': 60, 'A': 1, 'B': 47, 'C': 9, 'D': 40, 'E': 62, 'F': 59, 'G': 21, 'H': 32,
         'I': 70, 'J': 56, 'L': 15, 'M': 50, 'N': 63, 'O': 7, 'P': 31, 'R': 27, 'S': 39, 'T': 51, 'U': 25,
         'V': 29, 'W': 49, 'Y': 20, 'a': 61, 'b': 64, 'c': 44, 'd': 37, 'e': 16, 'f': 54, 'g': 11, 'h': 28,
         'i': 67, 'j': 17, 'k': 46, 'l': 3, 'm': 13, 'n': 42, 'o': 65, 'p': 41, 'q': 34, 'r': 10, 's': 19, 
        't': 36, 'u': 48, 'v': 53, 'w': 30, 'x': 4, 'y': 45, 'z': 35, '~': 0}
        pad_char='~'
        w_size = 30
        step_size = 5
        n_test_tuples = 128
        if w_size == -1:
            w_size = max(len(S) for S in sequences)
        X = []
        y = []

        steps = range(step_size, len(seq), step_size)
        if len(steps) == 0:
            steps = [len(seq)-1]
        for i in steps:
            end = i
            start = max(0, i-w_size)
            s = seq[start:end]
            assert len(s) <= w_size
            x = s.rjust(w_size, pad_char)
            assert len(x) == w_size
            X.append(x)
            y.append(seq[i])
            if len(X) > n_test_tuples:
                break

        test_from = X
        X_test, char2int = dni.tool.TwoDimEncoders.raw_to_bin_tensor(test_from, cust_char2int=char2int)
        return X_test
    
def generate_parsetree(inputdata):
    import warnings
    import copy
    import collections
    import time, timeit
    import random
    import numpy as np
    import nltk
    from nltk import Tree
    from nltk import PCFG,CFG
    from nltk.grammar import is_terminal, is_nonterminal, ProbabilisticProduction, Nonterminal
    from nltk.probability import DictionaryProbDist
    expr = inputdata
    gram = None
    with open('sql_full_XL.pcfg', 'r') as f:
        pcfg_string = f.read()
        from nltk import PCFG
        gram = PCFG.fromstring(pcfg_string)
    rules = [r for r in gram.productions() if r.prob() > 0]
    rule_index = collections.defaultdict(list)
    for r in rules:
        rule_index[r.lhs()].append(r)
    ix = rule_index
    len_ix = {lhs:len(rules) for lhs,rules in ix.items()}
    counter = 0
    term_index = {}
    new_rules = []
    for rule in gram.productions():
        # If Nonterm := Term or Nontern := Nonterm rule, skip
        if len_ix[rule.lhs()]==1 and len(rule.rhs())==1:
            new_rules.append(rule)
            continue
        # Otherwise creates a new rule
        new_rhs = []
        for r in rule.rhs():
            if is_nonterminal(r) :
                new_rhs.append(r)
            else:
                if r not in term_index:
                    new_left = Nonterminal('symb_'+str(r))
                    prule = ProbabilisticProduction(new_left,
                                                    [r],
                                                    prob=1.0)
                    term_index[r]=prule
                new_rhs.append(new_left)
        new_rule = ProbabilisticProduction(rule.lhs(),
                                           new_rhs,
                                           prob=rule.prob())
        new_rules.append(new_rule)
    new_rules += term_index.values()
    gram = PCFG(gram.start(), new_rules)
    rules = [r for r in gram.productions() if r.prob() > 0]
    rule_index = collections.defaultdict(list)
    for r in rules:
        rule_index[r.lhs()].append(r)
    old_grammar = gram
    # BFS
    arules = set()
    visited = set()
    Q = collections.deque()
    Q.append(gram.start())
    while len(Q) > 0:
        # Gets first symbol in queue
        symb = Q.popleft()
        # Fetches corresponding rules
        rules = rule_index[symb]
        for r in rules:
            arules.add(r)
            for symb in r.rhs():
                if is_nonterminal(symb) and not symb in visited:
                    Q.append(symb)
                    visited.add(symb)
    # Creates new rules
    arules = list(arules)
    gram = PCFG(old_grammar.start(), arules)

    symbols = set()
    for rule in gram.productions():
        symbols.add(rule.lhs())
        for r in rule.rhs():
            symbols.add(r)

    
    term_nodes = [S for S in symbols if is_terminal(S)]
    terminals = set(term_nodes)
    L = max(len(w) for w in terminals)
    def terminal(i, expr):
        for wid in reversed(range(1, L+1)):
            j = i + wid
            if j > len(expr):
                continue
            w = expr[i:j]
            if w in terminals:
                return w
        return None

    tokens = []
    index = []
    i = 0
    skipchars=['~']
    while i < len(expr):
        if not expr[i] in skipchars:
            term = terminal(i, expr)
            if term is None:
                if not ignore_errors:
                    raise ValueError('Could not match token', expr[i:])
                else:
                    print("null")
            tokens.append(term)
            index.append((i,len(term)))
            i += len(term)
        else:
            i += 1
    parser = nltk.EarleyChartParser(gram)
    parse = None
    for p in parser.parse(tokens):
        parse = p
        break

    rules = [r for r in gram.productions() if r.prob() > 0]
    rule_index = collections.defaultdict(list)
    for r in rules:
        rule_index[r.lhs()].append(r)
    # Gets the symbols BFS
    visited = [gram.start()]
    Q = collections.deque()
    Q.append(gram.start())

    while len(Q) > 0:
        # Gets first symbol in queue
        symb = Q.popleft()
        # Fetches corresponding rules
        rules = rule_index[symb]
        for r in rules:
            for symb in r.rhs():
                if is_nonterminal(symb) and not symb in visited:
                    Q.append(symb)
                    visited.append(symb)

    visited = list(reversed(visited))

    # Alternative method for checking correctness
    alt_visited = [S for S in symbols if is_nonterminal(S)]
    assert set(visited) == set(alt_visited)

    non_term_nodes = visited
    tree = parse
    n_tokens = len(tree.leaves())
    nt_symbols = non_term_nodes

    rule_feats = np.zeros((n_tokens, len(nt_symbols)))
    rule2feat = {s.symbol():i for i,s in enumerate(nt_symbols)}

    def visit(tree, offset):
        if isinstance(tree, Tree):
            n_tokens = 0
            for subtree in tree:
                n_tokens += visit(subtree, offset + n_tokens)
            symb = tree.label()
            j = rule2feat[symb]
            rule_feats[offset:offset+n_tokens, j] = 1
            return n_tokens
        else:
            return 1

    visit(tree, 0)
    seq = expr
    lex_index = index
    w_feats = rule_feats


    ch_len = len(seq)
    w_len  = w_feats.shape[0]
    n_feats = w_feats.shape[1]
    ch_feats = np.zeros((ch_len, n_feats))

    for i_w in range(w_len):
        f_w  = w_feats[i_w,...]
        i_ch,len_ch = lex_index[i_w]
        f_ch = np.tile(f_w, len_ch).reshape((len_ch, n_feats))
        ch_feats[i_ch:i_ch+len_ch,:] = f_ch
    return [ch_feats,nt_symbols]

# feature for "where" symbol in sql
def extractfeature1(intermiediate, inputdata):  
    import numpy as np
    tree = intermiediate[0]
    names = intermiediate[1]
    seq= inputdata
    pad_char='~'
    w_size = 30
    step_size = 5
    if w_size == -1:
        w_size = max(len(S) for S in sequences)
    X = []
    y = []
    prov_index = {}

    steps = range(step_size, len(seq), step_size)
    if len(steps) == 0:
        steps = [len(seq)-1]
    for i in steps:
        end = i
        start = max(0, i-w_size)
        s = seq[start:end]
        assert len(s) <= w_size
        x = s.rjust(w_size, pad_char)
        assert len(x) == w_size
        X.append(x)
        y.append(seq[i])
        prov_index[x] = (start, end)
    featuress = []
    for idx, name in enumerate(names):
        features = []
        for key, value in prov_index.items(): 
            start, end = value
            feature = tree[start:end,idx]
            L = end - start
            if L < 30:
                tmp = np.zeros((30))
                tmp[-L:] = feature
                feature = tmp
            features.append(feature)
        feat_name = 'F_' + name.symbol()
        featuress.append(features)
    return np.array(featuress[119])

# feature for "From" symbol in sql
def extractfeature2(intermiediate, inputdata):  
    import numpy as np
    tree = intermiediate[0]
    names = intermiediate[1]
    seq= inputdata
    pad_char='~'
    w_size = 30
    step_size = 5
    if w_size == -1:
        w_size = max(len(S) for S in sequences)
    X = []
    y = []
    prov_index = {}

    steps = range(step_size, len(seq), step_size)
    if len(steps) == 0:
        steps = [len(seq)-1]
    for i in steps:
        end = i
        start = max(0, i-w_size)
        s = seq[start:end]
        assert len(s) <= w_size
        x = s.rjust(w_size, pad_char)
        assert len(x) == w_size
        X.append(x)
        y.append(seq[i])
        prov_index[x] = (start, end)
    featuress = []
    for idx, name in enumerate(names):
        features = []
        for key, value in prov_index.items(): 
            start, end = value
            feature = tree[start:end,idx]
            L = end - start
            if L < 30:
                tmp = np.zeros((30))
                tmp[-L:] = feature
                feature = tmp
            features.append(feature)
        feat_name = 'F_' + name.symbol()
        featuress.append(features)
    return np.array(featuress[96])

registry = dni.udf.UDFRegistry.registry()
registry.add(generate_parsetree,"parsing_tree")
registry.add(extractfeature2,"from_symbol_feature")

In [4]:
clusterid = "10.128.0.51:6379"
AccessMethod= [[dni.access.LocalScanner,['sqldata.txt']]]
ActivationExt = ActivationExt
Neuron = [[[0],[0,1,2,3]]]
Models = ["track_history/models-03-2.78.hdf5"]
FeaturesFunctions = [[[extractfeature1,"from_symbol_feature"],"parsing_tree"]]
FeatureNames = ["WHERE","FROM"]
MetricName = ["Correlation"]

In [144]:

FeaturesFunctions

[[[<function __main__.extractfeature1(intermiediate, inputdata)>,
   'from_symbol_feature'],
  'parsing_tree']]

In [152]:
registry = dni.udf.UDFRegistry.registry()
for index1, elem1 in enumerate(FeaturesFunctions):
    for index2, elem2 in enumerate(elem1):
        if isinstance(elem2, list):
            for index3, elem3 in enumerate(elem2):
                if type(elem3) is str:
                    FeaturesFunctions[index1][index2][index3] = registry[elem3]
        else:
            if type(elem2) is str:
                FeaturesFunctions[index1][index2] = registry[elem2]

In [146]:
FeaturesFunctions

[[[<function __main__.extractfeature1(intermiediate, inputdata)>,
   <function __main__.extractfeature2(intermiediate, inputdata)>],
  <function __main__.generate_parsetree(inputdata)>]]

In [9]:
FeaturesFunctions[0][1]

<function __main__.generate_parsetree(inputdata)>

In [167]:
import dni
import importlib
importlib.reload(dni)

clusterid = "10.128.0.51:6379"
AccessMethod= [[dni.access.LocalScanner,'sqldata.txt']]
ActivationExt = ActivationExt
Neuron = [[[0],[0,1,2,3]]]
Models = ["track_history/models-03-2.78.hdf5"]
FeaturesFunctions = [[[extractfeature1,"from_symbol_feature"],"parsing_tree"]]
FeatureNames = ["WHERE","FROM"]
MetricName = ["Correlation"]


dni.inspect(clusterid = "10.128.0.51:6379",
            AccessMethod= AccessMethod,
            ActivationExt = ActivationExt, Neuron = [[[0],[0,1,2,3]]], 
            Models = ["track_history/models-03-2.78.hdf5"],
            FeaturesFunctions = [[[extractfeature1,extractfeature2],generate_parsetree]],
            FeatureNames = ["WHERE","FROM"],
            MetricName = ["Correlation"])

2019-11-25 22:20:48,858	WARNING worker.py:1268 -- WARNING: Not updating worker name since `setproctitle` is not installed. Install this with `pip install setproctitle` (or ray[debug]) to enable monitoring of worker processes.


(pid=3477) Using TensorFlow backend.
(pid=3477) WARNING:tensorflow:From /usr/local/lib/python3.5/dist-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
(pid=3477) Instructions for updating:
(pid=3477) If using Keras pass *_constraint arguments to layers.
(pid=3477) 2019-11-25 22:20:54.125244: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  AVX2 FMA
(pid=3477) To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
(pid=3477) 2019-11-25 22:20:54.135132: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
(pid=3477) 2019-11-25 22:20:54.135570: I tensorflow/compiler/xla/service/service.cc:168] XLA servic

In [8]:
clusterid = "10.128.0.51:6379"
AccessMethod= [[dni.access.LocalScanner,'sqldata.txt']]
ActivationExt = ActivationExt
Neuron = [[[0],[0,1,2,3]]]
Models = ["track_history/models-03-2.78.hdf5"]
FeaturesFunctions = [[[extractfeature1,extractfeature2],generate_parsetree]]
FeatureNames = ["WHERE","FROM"]
MetricName = ["Correlation"]
ray.shutdown()
ray.init(address=clusterid)

nummodel = len(Models)
layers = []
units = []
for i in range(len(Neuron)):
    layers.append(Neuron[i][0])
    units.append(Neuron[i][1])
maxlayer = 0
maxunits = 0
for i in range(len(Models)):
    maxlayer = max(maxlayer,len(layers[i]))
    maxunits = max(maxunits,len(units[i]))
numfeature = 0
for i in range(len(FeaturesFunctions)):
    if type(FeaturesFunctions[i]) is list and len(FeaturesFunctions[i]) == 2:
        numfeature += len(FeaturesFunctions[i][0])
    else:
        numfeature += 1
file_name = AccessMethod[0][1]
batch_size = 2


phy = dni.access.build_physical_scanner(dni.access.LocalScanner)
ps1 = phy.remote(file_name,batch_size)
phy_f_act = dni.feature.build_physical_feature_ext(dni.feature.FeatureExtractor)
fxt = phy_f_act.remote(ps1,FeaturesFunctions[0][1],FeaturesFunctions[0][0])
ps2 = phy.remote(file_name,batch_size)
phy_m_act = dni.model.build_physical_activation_ext(ActivationExt)
m = phy_m_act.remote(ps2,Models[0],layers[0],units[0])
corr_metric = dni.metric.IncrementalCorrelation(fxt,m,nummodel,maxlayer,maxunits,numfeature)
# corr_metric.open_itr()
# from prettytable import PrettyTable
# t = PrettyTable(['Model','Metric','Feature','Neuron','Score'])
# #for each model
# for i in range(len(Models)):
#     # for each layer
#     for j in range(len(layers[i])):
#         # for each unit
#         for k in range(len(units[i])):
#             # for each feature
#             for l in range(2):
#                 t.add_row([Models[i],MetricName[j],FeatureNames[l],"("+str(layers[i][j])+","+str(units[i][k])+")",corr_metric.extract(0,j,k,l)])               
# print(t)



2019-11-25 20:28:34,756	WARNING worker.py:1268 -- WARNING: Not updating worker name since `setproctitle` is not installed. Install this with `pip install setproctitle` (or ray[debug]) to enable monitoring of worker processes.
2019-11-25 20:28:34,787	WARNING actor.py:583 -- Actor is garbage collected in the wrong driver. Actor id = ActorID(cb5a56e514000000), class name = PhysicalScanner.
2019-11-25 20:28:34,814	WARNING actor.py:583 -- Actor is garbage collected in the wrong driver. Actor id = ActorID(6c3c451114000000), class name = PhysicalScanner.
2019-11-25 20:28:34,853	WARNING actor.py:583 -- Actor is garbage collected in the wrong driver. Actor id = ActorID(3cc9d99c14000000), class name = PhysicalActivationExt.
2019-11-25 20:28:34,854	WARNING actor.py:583 -- Actor is garbage collected in the wrong driver. Actor id = ActorID(b0fcf02d14000000), class name = PhysicalFeatureExt.


(pid=1142) Using TensorFlow backend.
(pid=1142) WARNING:tensorflow:From /usr/local/lib/python3.5/dist-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
(pid=1142) Instructions for updating:
(pid=1142) If using Keras pass *_constraint arguments to layers.
(pid=1142) 2019-11-25 20:28:37.632477: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  AVX2 FMA
(pid=1142) To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
(pid=1142) 2019-11-25 20:28:37.641466: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
(pid=1142) 2019-11-25 20:28:37.642125: I tensorflow/compiler/xla/service/service.cc:168] XLA servic

In [9]:
ray.get(ps1.has_next.remote())

True

In [13]:
import tables

ImportError: No module named 'tables'

In [163]:
import dni
import importlib
importlib.reload(dni)
importlib.reload(dni.feature)

<module 'feature' from '/home/zh2408/feature.py'>

In [ ]:
phy = dni.access.build_physical_scanner(dni.access.LocalScanner)
ps1 = phy.remote(file_name,batch_size)
#dynamicaaly build physical activation extractor based on logical activation extractor
def build_physical_activation_ext(LogicalActivationExt):
#     @ray.remote
    class PhysicalActivationExt(LogicalActivationExt):
        def get_next(self):
            if not self.has_next():
                return None
            input_table = ray.get(self.c.get_next.remote())
            table = dni.tool.HighDimensionPartitionableTable()
            # read the whole batch of inputs
            for num, input in input_table.itr():
                table.merge(dni.tool.activations_list_to_array(self.predict(input)))        
            return table
        def has_next(self):
            return super().has_next()
    return PhysicalActivationExt
phy_m_act = build_physical_activation_ext(ActivationExt)
m = phy_m_act(ps1,Models[0],layers[0],units[0])

In [164]:
phy = dni.access.build_physical_scanner(dni.access.LocalScanner)
ps1 = phy.remote(file_name,batch_size)
#dynamicaaly build physical activation extractor based on logical activation extractor
def build_physical_feature_ext(LogicalFeatureExt):
    class PhysicalFeatureExt(LogicalFeatureExt):
        def get_next(self):
            if not self.has_next():
                return None
            # implement same schema for table
            # implement the partition operator
            input_table = ray.get(self.c.get_next.remote())
            table = dni.tool.HighDimensionPartitionableTable()
            # read the whole batch of inputs
            for num, input in input_table.itr():
                table.merge(dni.tool.features_list_to_array(self.extract(input)))    
            return table
        def has_next(self):
            return super().has_next()
    return PhysicalFeatureExt
phy_f_act = build_physical_feature_ext(dni.feature.FeatureExtractor)
fxt = phy_f_act(ps1,FeaturesFunctions[0][1],FeaturesFunctions[0][0])

In [106]:
m = m.get_next()

In [165]:
k = fxt.get_next()

In [95]:
np.array(m).shape

(1, 54, 30, 4)

In [98]:
np.transpose(np.array(m), (1, 0, 3, 2))[..., np.newaxis,:].shape

(54, 1, 4, 1, 30)

In [40]:
np.transpose(np.array(k), (1, 0, 2)).shape

(54, 2, 30)

In [65]:
k.table[0][0][0][0][0].shape

(30,)

In [108]:
m.get_stat()

{'feature_num': 1,
 'input_num': 79,
 'layer_num': 1,
 'model_num': 1,
 'shape': (30,),
 'unit_num': 4}

In [166]:
k.get_stat()

{'feature_num': 2,
 'input_num': 79,
 'layer_num': 1,
 'model_num': 1,
 'shape': (30,),
 'unit_num': 1}

In [112]:
for i,j in m.itr(5):
    print(i,j.shape)

[5, 0, 0, 0, 0] (30,)
[5, 0, 0, 1, 0] (30,)
[5, 0, 0, 2, 0] (30,)
[5, 0, 0, 3, 0] (30,)


In [99]:
def activations_list_to_array(activations):
    activation_array = []
    for activation in np.transpose(np.array(activations), (1, 0, 3, 2))[..., np.newaxis,:]:
        activation_array.append([activation])
    return activation_array

In [101]:
np.array(activations_list_to_array(m)).shape

(54, 1, 1, 4, 1, 30)

In [117]:
import numpy as np
registry = dni.udf.UDFRegistry.registry()
registry.add(np.mean,"avg")
udf = registry["avg"]
udf([1,2,3])

2.0

In [118]:
registry = dni.udf.UDFRegistry.registry()
udf = registry["avg"]
udf([1,2,3])

2.0

In [20]:
dni.baseops.UnaryOp

baseops.UnaryOp

In [10]:
@ray.remote
class Scanner(dni.access.Scan):
    def __init__(self):
        self.f = open('sqldata.txt', 'r')
        self.f.seek(0)
        self.nextline = self.f.readline()
    def Next(self):
        if not self.HasMore():
            return ""
        currentline = self.nextline
        self.nextline = self.f.readline()
        return self.postprocess(currentline)
    def HasMore(self):
        if self.nextline:
            return True
        else:
            return False
    def postprocess(self,sttr):
        return sttr.split("\n")[0]

In [4]:
class BroadCaster(dni.baseops.UnaryOp):
    def get_next(self):
        # get tables from child and merge all of them
        if self.table == None:
            table = dni.tool.HighDimensionPartitionableTable()
            while ray.get(self.c.has_next.remote())
                table.merge(ray.get(self.feature_child.get_next.remote()))
            self.table = table
        # return tables to parent
        return self.table
    def has_next(self):
        return True

class Partitioner(dni.baseops.UnaryOp):
    def __init__(self, child, partition_number):
        super().__init__(child)
        self.partition_number = partition_number
        self.stack_of_partitions = []
    def get_next(self):
        # get table from child and partition it
        if len(self.stack_of_partitions) == 0:
            self.stack_of_partitions = ray.get(self.c.get_next.remote()).partition(self.partition_number)
        return self.stack_of_partitions.pop()
    def has_next(self):
        return ray.get(self.c.has_next.remote()) or len(self.stack_of_partitions) > 0

In [5]:
c = LocalScanner.remote('sqldata.txt')

In [169]:
True and False

False

In [6]:
if ray.get(c.have_next.remote()):
    print(ray.get(c.get_next.remote()))

 SELECT *, ix.* FROM )  SELECT pqohm.nl65p  AS bpxhg FROM yc11 WHERE p65.bn7<oo  AND ..false..<>.fj. ORDER BY ff DESC ) AS r08z union  SELECT .1875. * t8a5gu, false, wtl.* FROM vz AS rk4j6.* q36 AS w2orpr INNER JOIN l5  ON q3<=to4uao.tc WHERE f9c7kr<=azf8x GROUP BY  in7kmt$


In [11]:
Neuron = [[[0],[0,1,2,3]]]
Models = ["track_history/models-03-2.78.hdf5"]
clusterid = "10.128.0.51:6379"
Models = ["Models/boolean.h5"]
layers = []
units = []
for i in range(len(Neuron)):
    layers.append(Neuron[i][0])
    units.append(Neuron[i][1])

In [60]:
c = LocalScanner.remote('sqldata.txt')
m = ActivationExt.remote(c,"track_history/models-03-2.78.hdf5",layers[0],units[0])

(pid=19994) python3: ../nptl/pthread_mutex_lock.c:352: __pthread_mutex_lock_full: Assertion `INTERNAL_SYSCALL_ERRNO (e, __err) != ESRCH || !robust' failed.
(pid=20145) Using TensorFlow backend.
(pid=20145) WARNING:tensorflow:From /usr/local/lib/python3.5/dist-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
(pid=20145) Instructions for updating:
(pid=20145) If using Keras pass *_constraint arguments to layers.
(pid=20145) 2019-11-17 19:01:14.654785: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  AVX2 FMA
(pid=20145) To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
(pid=20145) 2019-11-17 19:01:14.661854: I tensorflow/core/platf

In [50]:
a = ray.get(m.have_next.remote())
a

True

In [61]:
a = ray.get(m.get_next.remote())
a

In [63]:
t = a.table

In [66]:
a.get_stat()


{'activation_shape': (30,), 'input_num': 54, 'layer_num': 1, 'unit_num': 4}

In [ ]:
new_table

In [ ]:
activation_per_layer

In [71]:
t[0][...,0].shape

(54, 30)

In [94]:
# a = np.array([[1, 2, 3], [4, 5, 6]])
# b = np.array([[9, 8, 7], [6, 5, 4]])
table1 = t
table2 = t
layer_num = len(table1)
unit_num = table1[0].shape[-1]

new_table = []
for i in range(layer_num):
    activation_per_layer = []
    for j in range(unit_num):
        activation_per_layer.append(np.concatenate((table1[i][...,j], table2[i][...,j])))
    print(np.array(activation_per_layer).shape)
    new_table.append(np.transpose(np.array(activation_per_layer), (1, 2, 0)))  
    
# np.concatenate((t[0][...,0], t[0][...,0])).shape
# activation_per_layer = []

(4, 108, 30)


In [96]:
layer_num = len(new_table)
unit_num = new_table[0].shape[-1]

In [98]:
layer_num 

1

In [14]:
class LocalScanner(Operator):
    def __init__(self, filename):
        self.f = open(filename, 'r')
        self.f.seek(0)
        self.nextline = self.f.readline()
    def get_next(self):
        if not self.have_next():
            return None
        currentline = self.nextline
        self.nextline = self.f.readline()
        return self.post_process(currentline)
    def have_next(self):
        if self.nextline:
            return True
        else:
            return False
    def post_process(self,sttr):
        return sttr.split("\n")[0]

#dynamicaaly build physical scanner based on logical scanner
def build_physical_scanner(LogicalScanner):
    @ray.remote
    class PhysicalScanner(LogicalScanner):
        def __init__(self,file,batch):
            super().__init__(file)
            self.batch = batch  
        def get_next(self):
            batch_inputs = []
            while self.have_next() and len(batch_inputs)<self.batch:
                batch_inputs.append(super().get_next())
            return InputTable(batch_inputs) 
        def have_next(self):
            return super().have_next()
    return PhysicalScanner

In [20]:
phy = build_physical_scanner(LocalScanner)
ps = phy.remote('sqldata.txt',2)
asd = ray.get(ps.get_next.remote())

In [21]:
ps = phy.remote('sqldata.txt',2)

In [22]:
asd = ray.get(ps.get_next.remote())

In [23]:
asd.get_stat()

{'input_num': 2}

In [248]:
sample_input = None
for i,j in asd.itr():
    sample_input = j

In [249]:
sample_input

' SELECT mwfl.* ORDER BY v1d.rk ASC ) o3py$'

In [118]:
table1 = t
table2 = t
layer_num = len(table1)
unit_num = table1[0].shape[-1]

In [119]:
t[0].shape

(54, 30, 4)

In [85]:
np.transpose(t[0], (2, 0, 1)).shape

(4, 54, 30)

In [193]:
table = ActivationTable()

In [194]:
table.get_stat()

'not initialized'

In [198]:
table.add_table(t)
table.get_stat()

{'activation_shape': (30,), 'input_num': 216, 'layer_num': 1, 'unit_num': 4}

In [58]:
a[0].shape

(25, 30, 4)

In [46]:
units

[[0, 1, 2, 3]]

In [28]:
# @ray.remote
class ActivationExt(Operator):
    def __init__(self, accessmethod, modelname, layer,unit):
        from keras.models import Model, load_model
        newmodel = load_model(modelname)
        newmodel._make_predict_function()
        outputs = [newmodel.layers[l].output for l in layer]
        self.model = Model(inputs = newmodel.input, outputs = outputs)
        self.unit = unit
        self.c = accessmethod
    def get_next(self):
        if not self.have_next():
            return None
        input = ray.get(self.c.get_next.remote())
        pred = self.predict(self,input)
        return ActivationTable(pred)
    def have_next(self):
        return ray.get(self.c.have_next.remote())
    def predict(self,input):
        input = self.preprocess(input)
        pred = self.model.predict(input)
        if not isinstance(pred, list):
            pred = [pred]
        for i in range(len(pred)):
            pred[i] = pred[i][...,self.unit]
        return pred
    def preprocess(self, input):
        seq = input
        char2int = {' ': 72, '$': 23, "'": 12, '(': 68, ')': 8, '*': 71, '+': 33, ',': 24, '-': 26, '.': 2,
         '/': 5, '0': 22, '1': 6, '2': 43, '3': 38, '4': 14, '5': 58, '6': 66, '7': 18, '8': 55, '9': 69,
         '<': 52, '=': 57, '>': 60, 'A': 1, 'B': 47, 'C': 9, 'D': 40, 'E': 62, 'F': 59, 'G': 21, 'H': 32,
         'I': 70, 'J': 56, 'L': 15, 'M': 50, 'N': 63, 'O': 7, 'P': 31, 'R': 27, 'S': 39, 'T': 51, 'U': 25,
         'V': 29, 'W': 49, 'Y': 20, 'a': 61, 'b': 64, 'c': 44, 'd': 37, 'e': 16, 'f': 54, 'g': 11, 'h': 28,
         'i': 67, 'j': 17, 'k': 46, 'l': 3, 'm': 13, 'n': 42, 'o': 65, 'p': 41, 'q': 34, 'r': 10, 's': 19, 
        't': 36, 'u': 48, 'v': 53, 'w': 30, 'x': 4, 'y': 45, 'z': 35, '~': 0}
        pad_char='~'
        w_size = 30
        step_size = 5
        n_test_tuples = 128
        if w_size == -1:
            w_size = max(len(S) for S in sequences)
        X = []
        y = []

        steps = range(step_size, len(seq), step_size)
        if len(steps) == 0:
            steps = [len(seq)-1]
        for i in steps:
            end = i
            start = max(0, i-w_size)
            s = seq[start:end]
            assert len(s) <= w_size
            x = s.rjust(w_size, pad_char)
            assert len(x) == w_size
            X.append(x)
            y.append(seq[i])
            if len(X) > n_test_tuples:
                break
        test_from = X
        X_test, char2int = dni.tool.TwoDimEncoders.raw_to_bin_tensor(test_from, cust_char2int=char2int)
        return X_test
    
#dynamicaaly build physical activation extractor based on logical activation extractor
def build_physical_activation_ext(LogicalActivationExt):
    @ray.remote
    class PhysicalActivationExt(LogicalActivationExt):
        def get_next(self):
            if not self.have_next():
                return None
            input_table = ray.get(self.c.get_next.remote())
            act_table = ActivationTable()
            # read the whole batch of inputs
            for num, input in input_table.itr():
                act_table.add_table(self.predict(input))    
            return act_table
        def have_next(self):
            return super().have_next()
    return PhysicalActivationExt

In [56]:
phy = build_physical_scanner(LocalScanner)
ps = phy.remote('sqldata.txt',2)
phyact = build_physical_activation_ext(ActivationExt)
m = phyact.remote(ps,"track_history/models-03-2.78.hdf5",layers[0],units[0])
actt = ray.get(m.get_next.remote())
actt.get_stat()

(pid=24010) Using TensorFlow backend.
(pid=24010) WARNING:tensorflow:From /usr/local/lib/python3.5/dist-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
(pid=24010) Instructions for updating:
(pid=24010) If using Keras pass *_constraint arguments to layers.
(pid=24010) 2019-11-17 21:43:52.446831: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  AVX2 FMA
(pid=24010) To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
(pid=24010) 2019-11-17 21:43:52.453956: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
(pid=24010) 2019-11-17 21:43:52.454222: I tensorflow/compiler/xla/service/service.cc:168] XL

{'activation_shape': (30,), 'input_num': 79, 'layer_num': 1, 'unit_num': 4}

{'activation_shape': (30,), 'input_num': 79, 'layer_num': 1, 'unit_num': 4}

In [139]:
c = LocalScanner.remote('sqldata.txt')
ray.get(c.have_next.remote())

True

In [148]:
c = LocalScanner.remote('sqldata.txt')
it = InputTable([ray.get(c.get_next.remote())])

(pid=21646) python3: ../nptl/pthread_mutex_lock.c:352: __pthread_mutex_lock_full: Assertion `INTERNAL_SYSCALL_ERRNO (e, __err) != ESRCH || !robust' failed.


In [149]:
it.get_stat()

{'input_num': 1}

In [152]:
it.add_table([ray.get(c.get_next.remote()),ray.get(c.get_next.remote())])
it.get_stat()

{'input_num': 4}

0  SELECT *, ix.* FROM )  SELECT pqohm.nl65p  AS bpxhg FROM yc11 WHERE p65.bn7<oo  AND ..false..<>.fj. ORDER BY ff DESC ) AS r08z union  SELECT .1875. * t8a5gu, false, wtl.* FROM vz AS rk4j6.* q36 AS w2orpr INNER JOIN l5  ON q3<=to4uao.tc WHERE f9c7kr<=azf8x GROUP BY  in7kmt$
1  SELECT 38.22300, opbu.*, tlf.z03n2w, i6fc.*, b0.ssey  AS ydfygj FROM d4 AS nufxa WHERE cn6f.ssi4<=71.07001 LIMIT 8519 OFFSET 691$


In [175]:
1 is not None

True

In [37]:
# @ray.remote
class FeatureExtractor():
    def __init__(self, accessmethod, intermediate_function, list_of_featurefunctions):
        self.intermediate = intermediate_function
        self.features = list_of_featurefunctions
        self.c = accessmethod
    def get_next(self):
        if not self.have_next():
            return None    
        input = ray.get(self.c.get_next.remote())    
        return self.extract(input)
    def extract(self, input):
        intermediate = self.intermediate(input)
        features_task = []
        # apply each feature function
        for featurefunction in self.features:
                features_task.append(ray.remote(featurefunction).remote(intermediate,input))
        features = []
        for i in range(len(features_task)):
                features.append(ray.get(features_task[i]))
        return features
    def have_next(self):
        return ray.get(self.c.have_next.remote())

In [50]:
#dynamicaaly build physical activation extractor based on logical activation extractor
def build_physical_feature_ext(LogicalFeatureExt):
    @ray.remote
    class PhysicalFeatureExt(LogicalFeatureExt):
        def get_next(self):
            if not self.have_next():
                return None
            input_table = ray.get(self.c.get_next.remote())
            f_table = dni.tool.FeatureTable()
            # read the whole batch of inputs
            for num, input in input_table.itr():
                f_table.add_table(self.extract(input))    
            return f_table
        def have_next(self):
            return super().have_next()
    return PhysicalFeatureExt

In [55]:
phy = build_physical_scanner(LocalScanner)
ps = phy.remote('sqldata.txt',2)
phy_f_act = build_physical_feature_ext(FeatureExtractor)
fxt = phy_f_act.remote(ps,generate_parsetree,[extractfeature1,extractfeature2])
ftt = ray.get(fxt.get_next.remote())
ftt.get_stat()

{'feature_num': 2, 'feature_shape': (30,), 'input_num': 79}

{'feature_num': 2, 'feature_shape': (30,), 'input_num': 79}

In [8]:
ll = [1,2,3,4]
for i in ll:
    print(i)

1
2
3
4


In [ ]:
f = FeatureExtractor(c1,generate_parsetree,[extractfeature1,extractfeature2])

In [254]:
inter = generate_parsetree(sample_input)

In [260]:
f = [extractfeature1(inter,sample_input),extractfeature2(inter,sample_input)]

In [293]:
ft = FeatureTable()

In [294]:
ft.get_stat()

'not initialized'

In [304]:
f = [extractfeature1(inter,sample_input),extractfeature2(inter,sample_input)]
ft.add_table(f)
ft.get_stat()

{'feature_num': 2, 'feature_shape': (30,), 'input_num': 80}

In [292]:
f[0].shape

(1024, 30)

In [ ]:
m = phyact.remote(ps,"track_history/models-03-2.78.hdf5",layers[0],units[0])
actt = ray.get(m.get_next.remote())
actt.get_stat()

In [265]:
f[0].shape

(8, 30)

In [ ]:
table1 = self.table
        if table1 is None:
            new_table = table2
        else:
            new_table = []
            for i in range(layer_num):
                activation_per_layer = []
                for j in range(unit_num):
                    activation_per_layer.append(np.concatenate((table1[i][...,j], table2[i][...,j])))
                new_table.append(np.transpose(np.array(activation_per_layer), (1, 2, 0))) 
        self.__init__(new_table)

In [266]:
for i in range(layer_num):
    self.table[i] = np.concatenate((self.table[0], newtable[0]))

(16, 30)

In [54]:
model_num = 1
layer_num = 1
unit_num = 4
feature_num = 2

In [88]:
# batch mode exhaustive correlation
class Incremental_Correlation():
    def __init__(self, feature_ext, model_ext, model_num, layer_num, unit_num, feature_num):
        import numpy as np
        self.sumXsquare = np.zeros(shape=(model_num, layer_num, unit_num, feature_num))
        self.sumYsquare = np.zeros(shape=(model_num, layer_num, unit_num, feature_num))
        self.sumX = np.zeros(shape=(model_num, layer_num, unit_num, feature_num))
        self.sumY = np.zeros(shape=(model_num, layer_num, unit_num, feature_num))
        self.sumXY = np.zeros(shape=(model_num, layer_num, unit_num, feature_num))
        self.n = np.full((model_num, layer_num, unit_num, feature_num),1e-17)
        self.model_child = model_ext
        self.feature_child = feature_ext
    def open_itr(self):
        while ray.get(self.model_child.have_next.remote()) is True and ray.get(self.feature_child.have_next.remote()) is True:
            feature_table = ray.get(self.feature_child.get_next.remote())
            feature_stat = feature_table.get_stat()
            activation_table = ray.get(self.model_child.get_next.remote())
            activation_stat = activation_table.get_stat()
            if(feature_stat["input_num"] != activation_stat["input_num"]):
                print(feature_stat["input_num"], activation_stat["input_num"])
            assert(feature_stat["input_num"] == activation_stat["input_num"])
            for i in range(feature_stat["input_num"]):
                for feature_num, feature in feature_table.itr(i):
                    for layer_num, unit_num, activation in activation_table.itr(i):
                        self.increment(feature,activation,0,layer_num,unit_num,feature_num)      
    # input should from one layer one unit
    def increment(self,input1,input2,model, layer, unit, feature):
        import numpy as np
        while input1.ndim != 1:
            input1 = np.concatenate(input1)
        while input2.ndim != 1:
            input2 = np.concatenate(input2)
        for i in range(len(input1)):
            self.sumXsquare[model, layer, unit, feature] += input1[i]**2
            self.sumYsquare[model, layer, unit, feature] += input2[i]**2
            self.sumX[model, layer, unit, feature] += input1[i]
            self.sumY[model, layer, unit, feature] += input2[i]
            self.sumXY[model, layer, unit, feature] += input1[i]*input2[i]
            self.n += 1
    def extract(self,model, layer, unit, feature):
        return (self.sumXY[model, layer, unit, feature] - self.sumX[model, layer, unit, feature]*self.sumY[model, layer, unit, feature]/\
                self.n[model, layer, unit, feature])/(((self.sumXsquare[model, layer, unit, feature] - self.sumX[model, layer, unit, feature]\
                **2/self.n[model, layer, unit, feature])**(1/2))*((self.sumYsquare[model, layer, unit, feature]\
                - self.sumY[model, layer, unit, feature]**2/self.n[model, layer, unit, feature])**(1/2))+1e-17)
    def get_class(self):
        return "Batch_Incremental_Comprehensive_Correlation"
    def get_name(self):
        return "Correlation"

In [111]:
# TODO support layers, units
import ray
import dni
import importlib
importlib.reload(dni)
import numpy as np
ray.shutdown()
ray.init(address="10.128.0.51:6379")

2019-11-17 22:30:01,816	WARNING worker.py:1268 -- WARNING: Not updating worker name since `setproctitle` is not installed. Install this with `pip install setproctitle` (or ray[debug]) to enable monitoring of worker processes.


{'node_ip_address': '10.128.0.51',
 'object_store_address': '/tmp/ray/session_2019-11-16_22-47-25_815266_4941/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2019-11-16_22-47-25_815266_4941/sockets/raylet',
 'redis_address': '10.128.0.51:6379',
 'session_dir': '/tmp/ray/session_2019-11-16_22-47-25_815266_4941',
 'webui_url': None}

In [104]:
phy = dni.access.build_physical_scanner(dni.access.LocalScanner)

2019-11-17 22:30:04,180	WARNING actor.py:583 -- Actor is garbage collected in the wrong driver. Actor id = ActorID(ec000fb708000000), class name = PhysicalScanner.
2019-11-17 22:30:04,200	WARNING actor.py:583 -- Actor is garbage collected in the wrong driver. Actor id = ActorID(035640c908000000), class name = PhysicalScanner.


(pid=26024) Using TensorFlow backend.
(pid=26024) WARNING:tensorflow:From /usr/local/lib/python3.5/dist-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
(pid=26024) Instructions for updating:
(pid=26024) If using Keras pass *_constraint arguments to layers.
(pid=26024) 2019-11-17 22:30:06.305075: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  AVX2 FMA
(pid=26024) To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
(pid=26024) 2019-11-17 22:30:06.311747: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
(pid=26024) 2019-11-17 22:30:06.312087: I tensorflow/compiler/xla/service/service.cc:168] XL

+-----------------------------------+-------------+---------+--------+----------------------+
|               Model               |    Metric   | Feature | Neuron |        Score         |
+-----------------------------------+-------------+---------+--------+----------------------+
| track_history/models-03-2.78.hdf5 | Correlation |  WHERE  | (0,0)  | 0.03992677222423158  |
| track_history/models-03-2.78.hdf5 | Correlation |   FROM  | (0,0)  | 0.10710977167558725  |
| track_history/models-03-2.78.hdf5 | Correlation |  WHERE  | (0,1)  | 0.053571646020853896 |
| track_history/models-03-2.78.hdf5 | Correlation |   FROM  | (0,1)  | 0.10934223866650748  |
| track_history/models-03-2.78.hdf5 | Correlation |  WHERE  | (0,2)  | -0.03795637676873332 |
| track_history/models-03-2.78.hdf5 | Correlation |   FROM  | (0,2)  | -0.04900551556588565 |
| track_history/models-03-2.78.hdf5 | Correlation |  WHERE  | (0,3)  | 0.04485307167406726  |
| track_history/models-03-2.78.hdf5 | Correlation |   FROM  

In [89]:
phy = dni.build_physical_scanner(LocalScanner)
ps1 = phy.remote('sqldata.txt',2)
phy_f_act = build_physical_feature_ext(FeatureExtractor)
fxt = phy_f_act.remote(ps1,generate_parsetree,[extractfeature1,extractfeature2])
# ftt = ray.get(fxt.get_next.remote())
# print(ftt.get_stat()["input_num"])
ps2 = phy.remote('sqldata.txt',2)
phy_m_act = build_physical_activation_ext(ActivationExt)
m = phy_m_act.remote(ps2,"track_history/models-03-2.78.hdf5",layers[0],units[0])
# actt = ray.get(m.get_next.remote())
# actt.get_stat()["input_num"]
corr_metric = IncrementalCorrelation(fxt,m,model_num,layer_num,unit_num,feature_num)
corr_metric.open_itr()

(pid=25286) Using TensorFlow backend.
(pid=25286) WARNING:tensorflow:From /usr/local/lib/python3.5/dist-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
(pid=25286) Instructions for updating:
(pid=25286) If using Keras pass *_constraint arguments to layers.
(pid=25286) 2019-11-17 21:55:04.786133: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  AVX2 FMA
(pid=25286) To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
(pid=25286) 2019-11-17 21:55:04.793072: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
(pid=25286) 2019-11-17 21:55:04.793447: I tensorflow/compiler/xla/service/service.cc:168] XL

+-----------------------------------+-------------+---------+--------+----------------------+
|               Model               |    Metric   | Feature | Neuron |        Score         |
+-----------------------------------+-------------+---------+--------+----------------------+
| track_history/models-03-2.78.hdf5 | Correlation |  WHERE  | (0,0)  | 0.014518562762868671 |
| track_history/models-03-2.78.hdf5 | Correlation |   FROM  | (0,0)  |  0.1291405753793342  |
| track_history/models-03-2.78.hdf5 | Correlation |  WHERE  | (0,1)  | 0.03177906175778336  |
| track_history/models-03-2.78.hdf5 | Correlation |   FROM  | (0,1)  |  0.1397928095421626  |
| track_history/models-03-2.78.hdf5 | Correlation |  WHERE  | (0,2)  | -0.04019260657924972 |
| track_history/models-03-2.78.hdf5 | Correlation |   FROM  | (0,2)  | -0.08666947121118071 |
| track_history/models-03-2.78.hdf5 | Correlation |  WHERE  | (0,3)  | 0.05281378493719408  |
| track_history/models-03-2.78.hdf5 | Correlation |   FROM  

In [97]:
units

[[0, 1, 2, 3]]

AssertionError: 

In [166]:
def x():
    return 1

In [169]:
ray.get(ray.remote(x).remote())

1

In [174]:
c1 = LocalScanner.remote('sqldata.txt')
f = FeatureExtractor.remote(c1,generate_parsetree,[extractfeature1,extractfeature2])
c2 = LocalScanner.remote('sqldata.txt')
m = ActivationExt.remote(c2,"track_history/models-03-2.78.hdf5",layers[0],units[0])
fff =ray.get(f.get_next.remote())
print(fff.get_stat())
mmm =ray.get(m.get_next.remote())
print(mmm.get_stat())

2019-11-17 03:39:31,539	WARNING actor.py:583 -- Actor is garbage collected in the wrong driver. Actor id = ActorID(4b48460002000000), class name = LocalScanner.
2019-11-17 03:39:31,555	WARNING actor.py:583 -- Actor is garbage collected in the wrong driver. Actor id = ActorID(cd1f5c7d02000000), class name = FeatureExtractor.
2019-11-17 03:39:31,557	WARNING actor.py:583 -- Actor is garbage collected in the wrong driver. Actor id = ActorID(20f84bc802000000), class name = LocalScanner.
2019-11-17 03:39:31,569	WARNING actor.py:583 -- Actor is garbage collected in the wrong driver. Actor id = ActorID(dbc0c02402000000), class name = ActivationExt.


(pid=9183) Using TensorFlow backend.
{'feature_shape': (30,), 'feature_num': 2, 'input_num': 54}
(pid=9183) WARNING:tensorflow:From /usr/local/lib/python3.5/dist-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
(pid=9183) Instructions for updating:
(pid=9183) If using Keras pass *_constraint arguments to layers.
(pid=9183) 2019-11-17 03:39:34.372146: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  AVX2 FMA
(pid=9183) To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
(pid=9183) 2019-11-17 03:39:34.379115: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
(pid=9183) 2019-11-17 03:39:34.379550: 

In [147]:
kkk=ray.get(f.get_next.remote())

In [183]:
kkk.get_stat()["input_num"]

54

In [175]:
zip([[1,2],[3,4]])

In [177]:
a = np.array([[1, 2], [3, 4]])


In [178]:
for index, x in np.ndenumerate(a):
    print(index, x)

(0, 0) 1
(0, 1) 2
(1, 0) 3
(1, 1) 4


In [185]:
for feature_num, feature in fff.get_features(0):
    print(feature_num, feature)

0 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]


In [180]:
mmm

{'feature_num': 2, 'feature_shape': (30,), 'input_num': 54}

{'activation_shape': (30,), 'input_num': 54, 'layer_num': 1, 'unit_num': 4}

In [48]:
class scc():
    def __init__(self,i):
        self.count = i
    
    def get_next(self):
        self.count += 1
        return self.count
    def have_next(self):
            return True

In [26]:
s =  scc()

In [30]:
s.get_next()

4

In [49]:
class LogicalScanner(AccessMethod):
    def __init__(self, file):
        self.file = file   
    def get_next(self):
        if self.have_next():
            return self.file.read_next()
        else:
            return None
    
    def have_next(self):
        return self.file.have_next()
            
class PhysicalScanner(LogicalScanner):
    def __init__(self,file,batch):
        super().__init__(data)
        self.batch = batch
    
    def get_next(self):
        batch_inputs = []
        while self.have_next() and len(batch_inputs)<self.batch:
            batch_inputs.append(super().get_next())
        return batch_inputs
    
    def have_next(self):
        return super().have_next()

In [50]:
ps = physical_scanner(5,2)

In [53]:
ps.get_next()

[10, 11]

In [250]:
def generate_parsetree(inputdata):
    import warnings
    import copy
    import collections
    import time, timeit
    import random
    import numpy as np
    import nltk
    from nltk import Tree
    from nltk import PCFG,CFG
    from nltk.grammar import is_terminal, is_nonterminal, ProbabilisticProduction, Nonterminal
    from nltk.probability import DictionaryProbDist
    expr = inputdata
    gram = None
    with open('sql_full_XL.pcfg', 'r') as f:
        pcfg_string = f.read()
        from nltk import PCFG
        gram = PCFG.fromstring(pcfg_string)
    rules = [r for r in gram.productions() if r.prob() > 0]
    rule_index = collections.defaultdict(list)
    for r in rules:
        rule_index[r.lhs()].append(r)
    ix = rule_index
    len_ix = {lhs:len(rules) for lhs,rules in ix.items()}
    counter = 0
    term_index = {}
    new_rules = []
    for rule in gram.productions():
        # If Nonterm := Term or Nontern := Nonterm rule, skip
        if len_ix[rule.lhs()]==1 and len(rule.rhs())==1:
            new_rules.append(rule)
            continue
        # Otherwise creates a new rule
        new_rhs = []
        for r in rule.rhs():
            if is_nonterminal(r) :
                new_rhs.append(r)
            else:
                if r not in term_index:
                    new_left = Nonterminal('symb_'+str(r))
                    prule = ProbabilisticProduction(new_left,
                                                    [r],
                                                    prob=1.0)
                    term_index[r]=prule
                new_rhs.append(new_left)
        new_rule = ProbabilisticProduction(rule.lhs(),
                                           new_rhs,
                                           prob=rule.prob())
        new_rules.append(new_rule)
    new_rules += term_index.values()
    gram = PCFG(gram.start(), new_rules)
    rules = [r for r in gram.productions() if r.prob() > 0]
    rule_index = collections.defaultdict(list)
    for r in rules:
        rule_index[r.lhs()].append(r)
    old_grammar = gram
    # BFS
    arules = set()
    visited = set()
    Q = collections.deque()
    Q.append(gram.start())
    while len(Q) > 0:
        # Gets first symbol in queue
        symb = Q.popleft()
        # Fetches corresponding rules
        rules = rule_index[symb]
        for r in rules:
            arules.add(r)
            for symb in r.rhs():
                if is_nonterminal(symb) and not symb in visited:
                    Q.append(symb)
                    visited.add(symb)
    # Creates new rules
    arules = list(arules)
    gram = PCFG(old_grammar.start(), arules)

    symbols = set()
    for rule in gram.productions():
        symbols.add(rule.lhs())
        for r in rule.rhs():
            symbols.add(r)

    
    term_nodes = [S for S in symbols if is_terminal(S)]
    terminals = set(term_nodes)
    L = max(len(w) for w in terminals)
    def terminal(i, expr):
        for wid in reversed(range(1, L+1)):
            j = i + wid
            if j > len(expr):
                continue
            w = expr[i:j]
            if w in terminals:
                return w
        return None

    tokens = []
    index = []
    i = 0
    skipchars=['~']
    while i < len(expr):
        if not expr[i] in skipchars:
            term = terminal(i, expr)
            if term is None:
                if not ignore_errors:
                    raise ValueError('Could not match token', expr[i:])
                else:
                    print("null")
            tokens.append(term)
            index.append((i,len(term)))
            i += len(term)
        else:
            i += 1
    parser = nltk.EarleyChartParser(gram)
    parse = None
    for p in parser.parse(tokens):
        parse = p
        break

    rules = [r for r in gram.productions() if r.prob() > 0]
    rule_index = collections.defaultdict(list)
    for r in rules:
        rule_index[r.lhs()].append(r)
    # Gets the symbols BFS
    visited = [gram.start()]
    Q = collections.deque()
    Q.append(gram.start())

    while len(Q) > 0:
        # Gets first symbol in queue
        symb = Q.popleft()
        # Fetches corresponding rules
        rules = rule_index[symb]
        for r in rules:
            for symb in r.rhs():
                if is_nonterminal(symb) and not symb in visited:
                    Q.append(symb)
                    visited.append(symb)

    visited = list(reversed(visited))

    # Alternative method for checking correctness
    alt_visited = [S for S in symbols if is_nonterminal(S)]
    assert set(visited) == set(alt_visited)

    non_term_nodes = visited
    tree = parse
    n_tokens = len(tree.leaves())
    nt_symbols = non_term_nodes

    rule_feats = np.zeros((n_tokens, len(nt_symbols)))
    rule2feat = {s.symbol():i for i,s in enumerate(nt_symbols)}

    def visit(tree, offset):
        if isinstance(tree, Tree):
            n_tokens = 0
            for subtree in tree:
                n_tokens += visit(subtree, offset + n_tokens)
            symb = tree.label()
            j = rule2feat[symb]
            rule_feats[offset:offset+n_tokens, j] = 1
            return n_tokens
        else:
            return 1

    visit(tree, 0)
    seq = expr
    lex_index = index
    w_feats = rule_feats


    ch_len = len(seq)
    w_len  = w_feats.shape[0]
    n_feats = w_feats.shape[1]
    ch_feats = np.zeros((ch_len, n_feats))

    for i_w in range(w_len):
        f_w  = w_feats[i_w,...]
        i_ch,len_ch = lex_index[i_w]
        f_ch = np.tile(f_w, len_ch).reshape((len_ch, n_feats))
        ch_feats[i_ch:i_ch+len_ch,:] = f_ch
    return [ch_feats,nt_symbols]

# feature for "where" symbol in sql
def extractfeature1(intermiediate, inputdata):  
    import numpy as np
    tree = intermiediate[0]
    names = intermiediate[1]
    seq= inputdata
    pad_char='~'
    w_size = 30
    step_size = 5
    if w_size == -1:
        w_size = max(len(S) for S in sequences)
    X = []
    y = []
    prov_index = {}

    steps = range(step_size, len(seq), step_size)
    if len(steps) == 0:
        steps = [len(seq)-1]
    for i in steps:
        end = i
        start = max(0, i-w_size)
        s = seq[start:end]
        assert len(s) <= w_size
        x = s.rjust(w_size, pad_char)
        assert len(x) == w_size
        X.append(x)
        y.append(seq[i])
        prov_index[x] = (start, end)
    featuress = []
    for idx, name in enumerate(names):
        features = []
        for key, value in prov_index.items(): 
            start, end = value
            feature = tree[start:end,idx]
            L = end - start
            if L < 30:
                tmp = np.zeros((30))
                tmp[-L:] = feature
                feature = tmp
            features.append(feature)
        feat_name = 'F_' + name.symbol()
        featuress.append(features)
    return np.array(featuress[119])

# feature for "From" symbol in sql
def extractfeature2(intermiediate, inputdata):  
    import numpy as np
    tree = intermiediate[0]
    names = intermiediate[1]
    seq= inputdata
    pad_char='~'
    w_size = 30
    step_size = 5
    if w_size == -1:
        w_size = max(len(S) for S in sequences)
    X = []
    y = []
    prov_index = {}

    steps = range(step_size, len(seq), step_size)
    if len(steps) == 0:
        steps = [len(seq)-1]
    for i in steps:
        end = i
        start = max(0, i-w_size)
        s = seq[start:end]
        assert len(s) <= w_size
        x = s.rjust(w_size, pad_char)
        assert len(x) == w_size
        X.append(x)
        y.append(seq[i])
        prov_index[x] = (start, end)
    featuress = []
    for idx, name in enumerate(names):
        features = []
        for key, value in prov_index.items(): 
            start, end = value
            feature = tree[start:end,idx]
            L = end - start
            if L < 30:
                tmp = np.zeros((30))
                tmp[-L:] = feature
                feature = tmp
            features.append(feature)
        feat_name = 'F_' + name.symbol()
        featuress.append(features)
    return np.array(featuress[96])

In [9]:
layers = []
units = []
for i in range(len(Neuron)):
    layers.append(Neuron[i][0])
    units.append(Neuron[i][1])

In [8]:
Neuron = [[[0],[0,1,2,3]]]
Models = ["track_history/models-03-2.78.hdf5"]
clusterid = "10.128.0.51:6379"
Models = ["Models/boolean.h5"]
# FeaturesFunctions = features 
# FeatureNames = feature_names
# FeatureExtractor = [F_Extractor]*6
# MetricExtractor = [corr_metric] 
# MetricName = Metric_names

In [44]:
layers = []
units = []
ray.shutdown()
ray.init(address=clusterid)
for i in range(len(Neuron)):
    layers.append(Neuron[i][0])
    units.append(Neuron[i][1])
    
maxlayer = 0
maxunits = 0
for i in range(len(Models)):
    maxlayer = max(maxlayer,len(layers[i]))
    maxunits = max(maxunits,len(units[i]))

numfeature = 0
for i in range(len(FeaturesFunctions)):
    if type(FeaturesFunctions[i]) is list and len(FeaturesFunctions[i]) == 2:
        numfeature += len(FeaturesFunctions[i][0])
    else:
        numfeature += 1

metrics = [None]*len(MetricName)
for i in range(len(MetricName)):
    if MetricName[i] == "Correlation":
        metrics[i] = dni.metric.Batch_Incremental_Comprehensive_Correlation(len(Models),maxlayer,maxunits,numfeature)

ModelActor = ActivationExt.remote(Models[0],layers[0])

FeatActor = [None]*len(FeaturesFunctions)
for i in range(len(FeaturesFunctions)):
    # if has intermediate result
    if type(FeaturesFunctions[i]) is list and len(FeaturesFunctions[i]) == 2:
        featureclass = ray.remote(dni.feature.CachedFeatureExtractor)
        FeatActor[i] = featureclass.remote(FeaturesFunctions[i][0],FeaturesFunctions[i][1])
    else:
        FeatActor[i] = FeatureExtractor[i].remote(FeaturesFunctions[i],FeatureNames[i])

ScannerActor = AccessMethod.remote()
act = []
feature = [[] for i in range(len(FeatureNames))]
while ray.get(ScannerActor.HasMore.remote()):
    data = ScannerActor.Next.remote()
    Ext = ModelActor.extract.remote(data,units[0])
    act.append(ray.get(Ext))
    currf = 0
    # for each group of feature functions
    for i in range(len(FeaturesFunctions)):
        if type(FeaturesFunctions[i]) is list and len(FeaturesFunctions[i]) == 2:
            FeatActor[i].setup.remote(data)
            # for each feature functions
            for j in range(len(FeaturesFunctions[i][0])):
                FeatActor[i].extract.remote(data,1)
                feature = ray.get(FeatActor[i].extract.remote(data,j))
                # for each layer
                for k in range(len(layers[0])):
                    # for each units
                    for l in range(len(units[i])):
                        # for each metric
                        for m in range(len(MetricName)):
                            metrics[i].increment(feature,ray.get(Ext)[k][...,l],0,k,l,currf)
                currf += 1
        else:
            feature[currf].append(ray.get(FeatActor[i].extract.remote(data)))
            currf += 1

from prettytable import PrettyTable
t = PrettyTable(['Model','Metric','Feature','Neuron','Score'])

#for each model
for i in range(len(Models)):
    # for each layer
    for j in range(len(layers[i])):
        # for each unit
        for k in range(len(units[i])):
            # for each feature
            for l in range(numfeature):
                t.add_row([Models[i],MetricName[j],FeatureNames[l],"("+str(layers[i][j])+","+str(units[i][k])+")",metrics[i].extract(0,j,k,l)])
                
print(t)


2019-11-11 20:04:38,109	WARNING worker.py:1268 -- WARNING: Not updating worker name since `setproctitle` is not installed. Install this with `pip install setproctitle` (or ray[debug]) to enable monitoring of worker processes.
2019-11-11 20:04:38,142	WARNING actor.py:583 -- Actor is garbage collected in the wrong driver. Actor id = ActorID(5d29414501000000), class name = Scanner.


(pid=3356) Using TensorFlow backend.
(pid=3356) WARNING:tensorflow:From /usr/local/lib/python3.5/dist-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
(pid=3356) Instructions for updating:
(pid=3356) If using Keras pass *_constraint arguments to layers.
(pid=3356) 2019-11-11 20:04:42.698390: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  AVX2 FMA
(pid=3356) To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
(pid=3356) 2019-11-11 20:04:42.707196: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
(pid=3356) 2019-11-11 20:04:42.707546: I tensorflow/compiler/xla/service/service.cc:168] XLA servic

(pid=2004) Using TensorFlow backend.
(pid=2004) WARNING:tensorflow:From /usr/local/lib/python3.5/dist-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
(pid=2004) Instructions for updating:
(pid=2004) If using Keras pass *_constraint arguments to layers.
(pid=2004) 2019-11-11 17:48:37.846780: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  AVX2 FMA
(pid=2004) To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
(pid=2004) 2019-11-11 17:48:38.353038: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
(pid=2004) 2019-11-11 17:48:38.353405: I tensorflow/compiler/xla/service/service.cc:168] XLA servic

In [ ]:
m = 

In [291]:
metric = []
#for each model
for i in range(len(Models)):
    metric_temp1 = []
    # for each layer
    for j in range(len(layers[i])):
        # for each unit
        for k in range(len(units[i])):
            # for each feature
            for l in range(len(FeatureNames)):
                
                
    
                

[Actor(CachedFeatureExtractor, c53598d002000000)]

1 4


In [316]:
# for each load of input data, each layer, get a number of activation, the last dimension is unit
act[0][0].shape

(54, 30, 4)

In [315]:
# for each feature, each load of input data, get a number of features
feature[0][0].shape

(54, 30)

+-----------------------------------+-------------+---------+--------+-----------------------+
|               Model               |    Metric   | Feature | Neuron |         Score         |
+-----------------------------------+-------------+---------+--------+-----------------------+
| track_history/models-03-2.78.hdf5 | Correlation |  WHERE  | (0,0)  |  0.02771628520146099  |
| track_history/models-03-2.78.hdf5 | Correlation |   FROM  | (0,0)  |  0.11961696236126476  |
| track_history/models-03-2.78.hdf5 | Correlation |  WHERE  | (0,1)  |  0.05012749261812874  |
| track_history/models-03-2.78.hdf5 | Correlation |   FROM  | (0,1)  |  0.12826532216877126  |
| track_history/models-03-2.78.hdf5 | Correlation |  WHERE  | (0,2)  |  -0.04503853262076593 |
| track_history/models-03-2.78.hdf5 | Correlation |   FROM  | (0,2)  | -0.038613080219511146 |
| track_history/models-03-2.78.hdf5 | Correlation |  WHERE  | (0,3)  |  0.059537094383411385 |
| track_history/models-03-2.78.hdf5 | Correlation 

In [ ]:



print(t)

In [229]:
s = Scanner.remote()

In [138]:
ray.get(s.HasMore.remote())

True

In [230]:
lines  = []
while ray.get(s.HasMore.remote()):
    lines.append(s.Next.remote())
ray.get(lines[0])

' SELECT *, ix.* FROM )  SELECT pqohm.nl65p  AS bpxhg FROM yc11 WHERE p65.bn7<oo  AND ..false..<>.fj. ORDER BY ff DESC ) AS r08z union  SELECT .1875. * t8a5gu, false, wtl.* FROM vz AS rk4j6.* q36 AS w2orpr INNER JOIN l5  ON q3<=to4uao.tc WHERE f9c7kr<=azf8x GROUP BY  in7kmt$'

In [145]:
line1 = ray.get(lines[0])

In [225]:
intermediate = generate_parsetree(ray.get(lines[1]))
intermediate[0].shape

(130, 171)

In [241]:
# for i,j in enumerate(intermediate[1]):
#     print(i,j)

In [240]:
f = extractfeature1(intermediate,ray.get(lines[1]))
f.shape

(25, 30)

In [256]:
featureext = ray.remote(dni.feature.CachedFeatureExtractor)

In [257]:
featureext = featureext.remote([extractfeature1,extractfeature2],generate_parsetree)

In [258]:
featureext.setup.remote(ray.get(lines[0]))
ray.get(featureext.extract.remote(lines[0],1))

ObjectID(ea7106654faf4b49c2340100000000c001000000)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

SyntaxError: can't assign to literal (<ipython-input-260-54a76c4220d1>, line 1)

In [177]:
Neuron = [[[0],[0,1,2,3]]]
Models = ["track_history/models-03-2.78.hdf5"]
clusterid = "10.128.0.51:6379"
AccessMethod= Scanner
ActivationExt = ActivationExt
Models = ["Models/boolean.h5"]
FeaturesFunctions = features 
FeatureNames = feature_names
FeatureExtractor = [F_Extractor]*6
MetricExtractor = [corr_metric] MetricName = Metric_names
layers = []
units = []
for i in range(len(Neuron)):
    layers.append(Neuron[i][0])
    units.append(Neuron[i][1])

In [186]:
ModelActor = ActivationExt.remote(Models[0],layers[0])
act1 = ray.get(ModelActor.extract.remote(lines[1],units[0]))
act1[0].shape

2019-11-10 20:16:22,983	WARNING worker.py:1619 -- The actor or task with ID ffffffffffffd1cda02401000000 is pending and cannot currently be scheduled. It requires {} for execution and {CPU: 1.000000} for placement, but this node only has remaining {object_store_memory: 2.832031 GiB}, {CPU: 4.000000}, {memory: 8.251953 GiB}. In total there are 0 pending tasks and 1 pending actors on this node. This is likely due to all cluster resources being claimed by actors. To resolve the issue, consider creating fewer actors or increase the resources available to this Ray cluster.


(pid=18136) Using TensorFlow backend.
(pid=18136) WARNING:tensorflow:From /usr/local/lib/python3.5/dist-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
(pid=18136) Instructions for updating:
(pid=18136) If using Keras pass *_constraint arguments to layers.
(pid=18136) 2019-11-10 20:16:25.174985: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  AVX2 FMA
(pid=18136) To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
(pid=18136) 2019-11-10 20:16:25.182514: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
(pid=18136) 2019-11-10 20:16:25.182985: I tensorflow/compiler/xla/service/service.cc:168] XL

(25, 30, 4)